In [2]:
import os
import re
import ast
import nltk 
import spacy
import pandas as pd
from tqdm import tqdm
from typing import List
from sklearn.preprocessing import LabelEncoder

c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP\projet\Recipe_generator\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
!python -m spacy download fr_core_news_lg

^C


     ------------------------------------ 571.8/571.8 MB 969.6 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')


In [3]:
nlp = spacy.load("fr_core_news_lg")

OSError: [E050] Can't find model 'fr_core_news_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\badje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\badje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\badje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\badje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\badje\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

True

In [5]:
current_dir = os.path.dirname(os.path.realpath("__file__"))
repo_dir = os.path.dirname(current_dir)
DATA_FILE = os.path.join(repo_dir,"data","recettes.csv")
data = pd.read_csv(DATA_FILE, encoding="utf-8",sep=',')

In [ ]:
data["Etapes"][0]

0        [{'Étape 1': "Enlever l'écorce et les pépins d...
1        [{'Étape 1': 'Pelez et émincez les oignons.'},...
2        [{'Étape 1': 'Couper la chair du potiron en gr...
3        [{'Étape 1': 'Couper le butternut en gros cube...
4        [{'Étape 1': 'Préchauffer le four à 210°C (the...
                               ...                        
47335    [{'Étape 1': "Transvaser le litre d'alcool dan...
47336    [{'Étape 1': 'Dans un shaker, mélanger tous le...
47337    [{'Étape 1': 'Commencer par réaliser un sirop ...
47338    [{'Étape 1': 'Verser les glaçons directement d...
47339    [{'Étape 1': 'Préparez directement dans un gra...
Name: Etapes, Length: 47340, dtype: object

In [6]:
def split_quantity_units(text):
    if re.match(r"^\d.*[a-zA-Z]$",text) is not None:
        my_list = re.split(r'(\d+)', text)
        return " ".join(my_list)
    return text

In [7]:
def rm_duplicated_dgt(text:str):
    if text.isnumeric() and len(text)%2==0 and text[:int(len(text)/2)]==text[int(len(text)/2):]:
        return(text[:int(len(text)/2)])
    else:
        return(text)

# rm_duplicated_dgt(text)

In [8]:
def spacy_prep(texts):
    cleaned_texts = []
    docs = list(nlp.pipe(texts))
    for doc in docs:
        cleaned_texts.append(" ".join([token.text for token in doc if not token.is_punct
                                                                            and not token.is_stop
                                                                            ]))
    return cleaned_texts

In [29]:
def join_qty_ingredients(ingredients:str):
    list_ingredients = ast.literal_eval(ingredients)
    new_ingredients = []
    for i in range(len(list_ingredients)-1):
        if bool(re.search(r'\d', list_ingredients[i])):
            list_ingredients[i] = rm_duplicated_dgt(list_ingredients[i])
            list_ingredients[i] = split_quantity_units(list_ingredients[i])
            if list_ingredients[i+1] != list_ingredients[i].split(" ")[-1] and list_ingredients[i+1] != "":
                new_ingredients.append(" ".join([list_ingredients[i],list_ingredients[i+1]]))
            else:
                if i+2 <= len(list_ingredients):
                    new_ingredients.append(" ".join([list_ingredients[i],list_ingredients[i+2]]))
    return "\n".join(new_ingredients)

In [1]:
def format_recipe(nom,ingredients,ustensiles,Etapes):
    text = f"{str(nom)}\nIngrédients : \n{str(ingredients)}\nUstensiles :\n{str(ustensiles)}\nInstructions :\n {str(Etapes)}"
    return(text)

In [21]:
data

,Nom,Duree,Personnes,Type,Ingredients,Ustensiles,Etapes
0,Soupe veloutée de potimarron et pommes de terre,45 min,4,Entrée,"['22', 'pommes de terre ', '22', 'oignons hach...","['louche', 'mixeur plongeant', 'Blender chauff...","[{'Étape 1': ""Enlever l'écorce et les pépins d..."
1,Soupe à l'oignon,45 min,4,Entrée,"['44', 'oignons ', '50g', 'de beurre ', '1c.à....","['louche', 'mixeur plongeant', 'Blender chauff...","[{'Étape 1': 'Pelez et émincez les oignons.'},..."
2,Velouté de potiron,1 h 15 min,4,Entrée,"['11', 'oignon ', '', 'poivre ', '', 'sel ', '...","['louche', 'mixeur plongeant', 'Couteau', 'Coc...",[{'Étape 1': 'Couper la chair du potiron en gr...
3,Velouté de butternut à la cannelle,35 min,4,Entrée,"['11', 'oignon ', '1c.à.s', 'c.à.s', 'de crème...","['louche', 'mixeur plongeant', 'Couteau', 'Cui...",[{'Étape 1': 'Couper le butternut en gros cube...
4,Tarte aux poireaux et aux lardons,50 min,6,Entrée,"['250g', 'de farine ', '140g', 'de beurre ', '...","['Moule à tarte', 'Four', 'couteau', 'pinceau'...",[{'Étape 1': 'Préchauffer le four à 210°C (the...
...,...,...,...,...,...,...,...
47335,Liqueur de Génépi,10 min,1,Boisson,"['40morceaux', 'de sucre ', '40brins', 'de gén...","['Entonnoir', 'balance de cuisine']","[{'Étape 1': ""Transvaser le litre d'alcool dan..."
47336,Caipirinha,3 min,1,Boisson,"['1dose', 'de jus de citron vert frais (15 ml)...","['Cuillère en bois', 'shaker']","[{'Étape 1': 'Dans un shaker, mélanger tous le..."
47337,Kombucha maison au citron et au gingembre,-,1,Boisson,"['1morceau', 'de gingembre râpé', '100g', 'de ...","['casserole', 'balance de cuisine']",[{'Étape 1': 'Commencer par réaliser un sirop ...
47338,Gin tonic,5 min,1,Boisson,"['4cl', 'de gin ', '2rondelles', 'de citron ',...",[],[{'Étape 1': 'Verser les glaçons directement d...


In [10]:
unit_abbreviation = {
    "c.à.s" : "cuillère à soupe",
    "c.à.c" : "cuillère à café",
    "g"     : "gramme",
    "l"     : "litre",
    "cl"    : "centilitre",
    "ml"    : "millilitre",
    "kg"    : "kilogramme",
}

In [11]:
print(join_qty_ingredients(data["Ingredients"][20]))

 1 kg de champignon « trompettes de la mort »,  2 c.à.s d'huile d'olive ,  1 gousse d'ail hachée,  1 ⁄ 2 botte de persil haché


In [12]:
def remove_accents(text:str):
    text = re.sub(r'[àáâãäå]', 'a', text)
    text = re.sub(r'[èéêë]', 'e', text)
    text = re.sub(r'[ìíîï]', 'i', text)
    text = re.sub(r'[òóôõö]', 'o', text)
    text = re.sub(r'[ùúûü]', 'u', text)
    return(text)

In [32]:
len(data)

47340

In [34]:
TEXT_FEATURES = ["Nom","Ingredients","Ustensiles","Etapes"]
nom,ingredients,ustensiles,Etapes = (data[key][0] for key in TEXT_FEATURES)

In [40]:
df.iloc["recipe"][0] = "EREJOEfj"

C:\Users\badje\AppData\Local\Temp\ipykernel_38852\642698640.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["recipe"][0] = "EREJOEfj"


In [57]:
def preprocessing_fn(df:pd.DataFrame):
    data = df.copy()
    TEXT_FEATURES = ["Nom","Ingredients","Ustensiles","Etapes"]
    CATEGORICAL_FEATURES = ["Type"]
    
    for key in tqdm(TEXT_FEATURES):
        data[key] = data[key].apply(lambda x:x.lower())
        if key == "Ingredients":
            data[key] = data[key].apply(lambda x:join_qty_ingredients(x))
        data[key] = spacy_prep(data[key].to_numpy())
        data[key] = data[key].apply(lambda x: remove_accents(x))

    recipes = []
    for i in tqdm(range(len(data))):
        nom,ingredients,ustensiles,Etapes = (data[key][i] for key in TEXT_FEATURES)
        recipes.append(format_recipe(nom,ingredients,ustensiles,Etapes))
    data["recettes"] = recipes

    for key in tqdm(CATEGORICAL_FEATURES):
        features =data[key].unique()
        encoder = LabelEncoder()
        encoder.fit(features)
        data[key] = encoder.transform(data["Type"])

   
    return data

In [58]:
cleaned_data = preprocessing_fn(data)

100%|██████████| 1/1 [00:00<00:00, 34.66it/s]


In [59]:
cleaned_data

,Nom,Duree,Personnes,Type,Ingredients,Ustensiles,Etapes,recettes
0,soupe veloute potimarron pomme terre,45 min,4,4,2 pomme terre 2 oignon hache 3 gousse ail ha...,louche mixeur plonger blender chauffer couteau...,etape 1 enlever ecorce pepin potimarron obliga...,soupe veloute potimarron pomme terre 2 pomme ...
1,soupe oignon,45 min,4,4,4 oignon 50 gramme beurre 1 c.a. huile 1...,louche mixeur plonger blender chauffer saladie...,etape 1 pelez emincer oignon etape 2 faire rev...,soupe oignon 4 oignon 50 gramme beurre 1 ...
2,veloute potiron,1 h 15 min,4,4,1 oignon 1 quartier potiron 1 brique creme f...,louche mixeur plonger couteau cocotte fonte po...,etape 1 couper chair potiron gros de etape 2 c...,veloute potiron 1 oignon 1 quartier potiron...
3,veloute butternut cannelle,35 min,4,4,1 oignon 1 c.a. creme frais 1 pincee canne...,louche mixeur plonger couteau cuillere bois po...,etape 1 couper butternut gros cube etape 2 cou...,veloute butternut cannelle 1 oignon 1 c.a. ...
4,tarte poireau lardon,50 min,6,4,250 gramme farine 140 gramme beurre 2 c....,moule tarte four couteau pinceau rape poele sa...,etape 1 prechauffer four 210 degre c thermosta...,tarte poireau lardon 250 gramme farine 14...
...,...,...,...,...,...,...,...,...
47335,liqueur genepi,10 min,1,2,40 morceau sucre 40 brin genepi 1 l alco...,entonnoir balance cuisine,etape 1 transvaser litre alcool gros bouteille...,liqueur genepi 40 morceau sucre 40 brin g...
47336,caipirinha,3 min,1,2,1 dose jus citron vert frais 15 millilitre j...,cuillere bois shaker,etape 1 shaker melanger ingredient glace pilee...,caipirinha 1 dose jus citron vert frais 15 ...
47337,kombucha maison citron gingembre,-,1,2,1 morceau gingembre raper 100 gramme sucre...,casserole balance cuisine,etape 1 commencer realiser sirop chauffer eau ...,kombucha maison citron gingembre 1 morceau ...
47338,gin tonic,5 min,1,2,4 cl gin 2 rondelle citron 8 cl tonic,,etape 1 verser glaçon verre etape 2 ajouter gi...,gin tonic 4 cl gin 2 rondelle citron 8 ...


In [60]:
cleaned_data.to_csv(os.path.join(repo_dir,"data","cleaned_data.csv"),encoding="utf-8",sep=",", index=False)